# **1. Project Overview**

# **Project Title: Visual Shopping Assistant with Multimodal RAG**
A multimodal RAG (Retrieval-Augmented Generation) system that is capable of responding to user product-related questions with a blend of:

Product text data (title, specs, reviews)

Product images

This enables the system to perceive product features both visually and textually.

# Dataset Loading  


| File Name              | Contents                                          |
| ---------------------- | ------------------------------------------------- |
| `description.csv.xlsx` | Titles, brief product descriptions                |
| `image.csv.xlsx`       | Product IDs and image links                       |
| `style.csv.xlsx`       | Metadata like gender, article type, color         |
| `document.csv.xlsx`    | Long-form text like user reviews, Q\&A, specs     |
| JSON files (`*.json`)  | Extra metadata like brand, price, and detail info |


 Step-by-Step Data Loading & Merging

 Step 1: Read CSVs



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import os
data_path = "/content/drive/MyDrive/custom"
description=pd.read_excel("/content/drive/MyDrive/custom/description.csv.xlsx")
images=pd.read_excel("/content/drive/MyDrive/custom/image.csv.xlsx")
style=pd.read_excel("/content/drive/MyDrive/custom/style.csv.xlsx")
documentation=pd.read_excel("/content/drive/MyDrive/custom/document.csv.xlsx")

print("description data:", description.shape)
print("image data:", images.shape)
print("style data:", style.shape)
print("documentation data:", documentation.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
description data: (51, 3)
image data: (51, 3)
style data: (51, 10)
documentation data: (51, 5)


Step 2: Merge Them



This merges everything on the id column (Product ID). You now have one table combine with:

Product ID

Product title & description

Image file name

Metadata like category, gender, color

Reviews, Q&A, specifications



In [ ]:

description['Product ID'] = description['Product ID'].astype(str)
images['Product ID'] = images['Product ID'].astype(str)
style['Product ID'] = style['Product ID'].astype(str)
documentation['Product ID'] = documentation['Product ID'].astype(str)
combine = description.merge(images, on='Product ID', how='outer') \
                       .merge(style, on='Product ID', how='outer') \
                       .merge(documentation, on='Product ID', how='outer')
combine.to_excel("/content/drive/MyDrive/custom/merged_products.xlsx", index=False)
combine.head()
combine_2=combine.drop(columns="Product_ID _y")
print(combine_2.head())

  Product_ID _x                                            deatil  Product ID  \
0      100.json  The Samsung Galaxy F06 5G is one of the most a...        100   
1      101.json  The **KRISONS Beatz 1000W Bluetooth Home Audio...        101   
2      102.json  \n\nThe **Panasonic 20 L Solo Microwave Oven (...        102   
3      103.json  \nThe **LG 7 kg Fully Automatic Front Load Was...        103   
4      104.json  The Pikham Woven Kanjivaram Silk Blend Jacquar...        104   

                                                link  Gender Primary Category  \
0  https://www.flipkart.com/samsung-galaxy-f06-5g...  Unisex      Electronics   
1  https://www.flipkart.com/krisons-beatz-fm-usb-...  Unisex      Electronics   
2  https://www.flipkart.com/panasonic-20-l-epoxy-...  Unisex      Electronics   
3  https://www.flipkart.com/lg-7-kg-5-star-invert...  Unisex  Home Appliances   
4  https://www.flipkart.com/pikham-woven-kanjivar...   Women         Clothing   

  Secondary Category      

Step 3: Add JSON Data

Each product has a corresponding JSON file (e.g. product_10001.json) which contains:

brand, price, detail, and other product-specific structured fields

In [ ]:
import os
import json
import pandas as pd



json_folder = '/content/drive/MyDrive/custom/meta image'

json_data_dict = {}

for file in os.listdir(json_folder):
    if file.endswith('.json'):
        file_path = os.path.join(json_folder, file)
        with open(file_path, 'r') as f:
            data = json.load(f)
            product_id = file.replace('.json', '')
            json_data_dict[product_id] = data

In [ ]:
def get_json_info(Product_ID):

    Product_ID = str(Product_ID)
    if Product_ID in json_data_dict:
        json_content = json_data_dict[Product_ID]
        texts = []

        if isinstance(json_content, dict):
            for key, val in json_content.items():
                if isinstance(val, list):
                    texts.extend([str(item) for item in val])
                elif isinstance(val, (str, dict, list)):
                    texts.append(str(val))
        elif isinstance(json_content, list):
            for item in json_content:
                if isinstance(item, (str, dict, list)):
                    texts.append(str(item))

        combined_text = ' '.join(texts)
        return combined_text
    else:
        return ''

# Add a new column to CSV DataFrame with merged JSON text info
combine_2['json_text'] = combine_2['Product ID'].apply(get_json_info)

# Step 5: Save the updated DataFrame to new CSV
combine_2.to_csv('/content/drive/MyDrive/custom/merged_products_with_json.csv', index=False)

print("Merged CSV with JSON data saved successfully!")

Merged CSV with JSON data saved successfully!


Now, each row in combine has a json_text column that is a concatenation of:

Product description

Metadata

JSON detail

Specs, Q&A, reviews



Step 4: Save Final Merged Data


In [ ]:
print(combine_2.head(1))

  Product_ID _x                                            deatil  Product ID  \
0      100.json  The Samsung Galaxy F06 5G is one of the most a...        100   

                                                link  Gender Primary Category  \
0  https://www.flipkart.com/samsung-galaxy-f06-5g...  Unisex      Electronics   

  Secondary Category ArticleType Base Colour Season  Year     Usage  \
0        Smartphones  Smartphone      Violet    All  2025  Personal   

                                  productDisplayName  \
0  Samsung Galaxy F06 5G (Lit Violet, 128 GB) (4 ...   

                                        Product Name  \
0  Samsung Galaxy F06 5G (Lit Violet, 128 GB) (4 ...   

                                 Product Description  \
0  Highest 5G Bands, 25W fast charging, 5000mAh b...   

                                        User Reviews  \
0  Excellent display, smooth performance, impress...   

                                     Q&A Threads  \
0  Q: Is anirod version 14A

1. Text Chunking & Document Creation
2. Embedding: Text, Image, and Multimodal
3. Vector Indexing with Qdrant
4. Query Flow (Embedding + Retrieval + RAG)

🔹 1. TEXT CHUNKING & DOCUMENT CREATION


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document  # Import Document from langchain
import pandas as pd


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Remove the import from llama_index.core.schema as it's not needed for this part
# from llama_index.core.schema import Document


docs = []
for index, row in combine_2.iterrows():
    # Extract all relevant metadata fields
    metadata = {
        "Product_ID_x": str(row.get("Product_ID _x", "")),
        "Product_ID_y": str(row.get("Product_ID _y", "")),
        "Product ID": str(row.get("Product ID", "")),
        "Detail": row.get("deatil", ""),
        "Link": row.get("link", ""),
        "Gender": row.get("Gender", ""),
        "Primary Category": row.get("Primary Category", ""),
        "Secondary Category": row.get("Secondary Category", ""),
        "ArticleType": row.get("ArticleType", ""),
        "Base Colour": row.get("Base Colour", ""),
        "Season": row.get("Season", ""),
        "Year": row.get("Year", ""),
        "Usage": row.get("Usage", ""),
        "Product Display Name": row.get("productDisplayName", ""),
        "Product Name": row.get("Product Name", ""),
        "Product Description": row.get("Product Description", ""),
        "User Reviews": row.get("User Reviews", ""),
        "QA Threads": row.get("Q&A Threads", ""),
    }

    # Content to embed — can be just json_text or also a readable summary
    page_text = f"{row.get('json_text', '')}"

    # Create Document using the langchain Document class
    doc = Document(
        page_content=page_text,
        metadata=metadata
    )
    docs.append(doc)

# Now this line will work correctly because the documents have the 'page_content' attribute
docs = text_splitter.split_documents(docs)
print(docs)

[Document(metadata={'Product_ID_x': '100.json', 'Product_ID_y': '', 'Product ID': '100', 'Detail': '', 'Link': 'https://www.flipkart.com/samsung-galaxy-f06-5g-lit-violet-128-gb/p/itm140da3412b73b?pid=MOBH9AS4FA5RUHSY&lid=LSTMOBH9AS4FA5RUHSYIYVIFO&marketplace=FLIPKART&fm=neo%2Fmerchandising&iid=M_32f39bc2-1a3a-434c-a50d-8ea0817d6921_1_1BUWY8OBA8L9_MC.MOBH9AS4FA5RUHSY&ppt=browse&ppn=browse&ssid=o9hsmo7als0000001747954534726&otracker=clp_pmu_v2_Latest%2BSamsung%2Bmobiles%2B_1_1.productCard.PMU_V2_SAMSUNG%2BGalaxy%2BF06%2B5G%2B%2528Lit%2BViolet%252C%2B128%2BGB%2529_samsung-mobile-store_MOBH9AS4FA5RUHSY_neo%2Fmerchandising_0&otracker1=clp_pmu_v2_PINNED_neo%2Fmerchandising_Latest%2BSamsung%2Bmobiles%2B_LIST_productCard_cc_1_NA_view-all&cid=MOBH9AS4FA5RUHSY', 'Gender': 'Unisex', 'Primary Category': 'Electronics', 'Secondary Category': 'Smartphones', 'ArticleType': 'Smartphone', 'Base Colour': 'Violet', 'Season': 'All', 'Year': 2025, 'Usage': 'Personal', 'Product Display Name': 'Samsung Galaxy

In [ ]:
!pip install langchain-huggingface
!pip install langchain-community
!pip install qdrant-client # Make sure qdrant-client is installed


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 9.0 MB/s eta 0:00:00


 2. TEXT, IMAGE, AND MULTIMODAL EMBEDDING
You now convert both text and image into vectors (embeddings) so that they can be stored and retrieved using similarity search.



Text Embedding (using SentenceTransformer or Hugging Face)


In [ ]:
!pip install langchain-huggingface
!pip install langchain-community
!pip install qdrant-client # Make sure qdrant-client is installed

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient # Import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct # Import necessary models

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print(embedding_model)
# 3. Generate embeddings
# Instead of using the undefined 'texts' variable, extract the page_content from the 'docs'
embeddings = [embedding_model.embed_query(doc.page_content) for doc in docs]

# 4. Initialize Qdrant client
# Re-initialize the client as it might have been used for image data in a previous cell
client = QdrantClient(path="qdrant_db")  # local Qdrant vector DB path

collection_name = "text_collection"

# 5. Create collection if not exists
# Check if embeddings is not empty before accessing embeddings[0]
if embeddings and not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=len(embeddings[0]), distance=Distance.COSINE)
    )
elif embeddings:
    # If collection exists, ensure the vector size matches if necessary
    collection_info = client.get_collection(collection_name=collection_name)
    if collection_info.vectors_config.params_map['default'].size != len(embeddings[0]):
         print(f"Warning: Existing collection '{collection_name}' has different vector size. Consider dropping and recreating.")


# 6. Prepare points for upsert
points = []
# Iterate through docs and their corresponding embeddings
for idx, (doc, vector) in enumerate(zip(docs, embeddings)):
    # Include metadata in the payload if desired
    payload = {"text": doc.page_content, "media_type": "text"}
    # Add all metadata from the document to the payload
    payload.update(doc.metadata)

    points.append(
        PointStruct(id=idx, vector=vector, payload=payload)
    )

# 7. Upsert points into Qdrant
# Only upsert if there are points to add
if points:
    client.upsert(collection_name=collection_name, points=points)
    print(f"Stored {len(points)} text embeddings in collection '{collection_name}'")
else:
    print("No text documents to store embeddings for.")

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False
Stored 143 text embeddings in collection 'text_collection'


In [ ]:
from PIL import Image
from io import BytesIO
import pandas as pd
import os

image_data = []
df = pd.DataFrame() # Initialize an empty DataFrame if it's not already defined

# Assuming 'Product ID' column contains the image filenames
for index, row in images.iterrows():
  image_filename = str(row['Product ID']) + '.png' # Assuming image files are .jpg and named after Product ID
  path = "/content/drive/MyDrive/custom/images/" + image_filename
  try:
      with open(path, "rb") as file:
            img_bytes = BytesIO(file.read())
            extracted_image = Image.open(img_bytes).resize((256, 256))
            image_data.append([extracted_image])
            new_row = {'path': path,
                       'media_type': "image",
                       'embeddings': ""}
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
  except FileNotFoundError:
      print(f"Warning: Image file not found at {path}")
  except Exception as e:
      print(f"Error processing image at {path}: {e}")

In [ ]:
from PIL import Image
from io import BytesIO
import pandas as pd
import os

image_data = []
df = pd.DataFrame() # Initialize an empty DataFrame if it's not already defined

# Assuming 'Product ID' column contains the image filenames
for index, row in images.iterrows():
  image_filename = str(row['Product ID']) + '.png' # Assuming image files are .jpg and named after Product ID
  path = "/content/drive/MyDrive/custom/images/" + image_filename
  try:
      with open(path, "rb") as file:
            img_bytes = BytesIO(file.read())
            extracted_image = Image.open(img_bytes).resize((256, 256))
            image_data.append([extracted_image])
            new_row = {'path': path,
                       'media_type': "image",
                       'embeddings': ""}
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
  except FileNotFoundError:
      print(f"Warning: Image file not found at {path}")
  except Exception as e:
      print(f"Error processing image at {path}: {e}")

In [ ]:
print(df.head())

                                           path media_type embeddings
0  /content/drive/MyDrive/custom/images/100.png      image           
1  /content/drive/MyDrive/custom/images/101.png      image           
2  /content/drive/MyDrive/custom/images/102.png      image           
3  /content/drive/MyDrive/custom/images/103.png      image           
4  /content/drive/MyDrive/custom/images/104.png      image           


 Image Embedding (using CLIP)


In [ ]:
# Install transformers library if not already installed
!pip install transformers

# Import CLIPModel and CLIPProcessor from the transformers library
from transformers import CLIPModel, CLIPProcessor
from PIL import Image
import torch

# Load CLIP model
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
image_embeddings = []

for index, row in df.iterrows():
    try:
        image_path = row["path"]
        image = Image.open(image_path).convert("RGB")

        inputs = clip_processor(images=image, return_tensors="pt")
        with torch.no_grad():
            embedding = clip_model.get_image_features(**inputs)

        # Normalize embedding
        embedding = torch.nn.functional.normalize(embedding, p=2, dim=1)

        # Convert to list
        embedding_np = embedding[0].cpu().numpy().tolist()
        image_embeddings.append(embedding_np)

        # Save in dataframe
        df.at[index, 'embeddings'] = embedding_np

    except Exception as e:
        print(f"Failed at {image_path} because {e}")


In [ ]:
!pip install qdrant-client llama-index
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance

# Connect to Qdrant
client = QdrantClient(path="qdrant_mm_db")

# Create collection for images
client.recreate_collection(
    collection_name="image_collection",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE)  # 512 = CLIP embedding size
)

# Insert points into Qdrant
points = []
for idx, row in df.iterrows():
    points.append(
        PointStruct(
            id=idx,
            vector=row['embeddings'],
            payload={"path": row["path"], "media_type": "image"}
        )
    )

client.upsert(collection_name="image_collection", points=points)



INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 12.2 MB/s eta 0:00:00


<ipython-input-19-4ab0cfde6c35>:9: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

 Multimodal Embedding (text & image in same space)


🔹 3. VECTOR INDEXING IN QDRANT


In [ ]:
from sentence_transformers import SentenceTransformer
from PIL import Image
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance

# 1. Load CLIP multi-modal model (outputs 512-dim embeddings)
model = SentenceTransformer('clip-ViT-B-32')

# 2. Prepare your text documents and image paths
texts = [doc.page_content for doc in docs]  # aapke text docs
image_paths = df['path'].tolist()           # aapke image file paths in pandas DataFrame 'df'

# 3. Generate embeddings
text_embeddings = model.encode(texts, convert_to_tensor=False)
image_embeddings = []
for path in image_paths:
    image = Image.open(path).convert("RGB")
    emb = model.encode(image, convert_to_tensor=False)
    image_embeddings.append(emb)

# 4. Initialize Qdrant client
# The client with path="qdrant_db" is already initialized in a previous cell.
# Do not re-initialize it here.
# client = QdrantClient(path="qdrant_db")  # local Qdrant DB path

collection_name = "multimodal_collection"

# 5. Create collection if not exists (512 dimension, cosine distance)
if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=512, distance=Distance.COSINE)
    )

# 6. Prepare points for upsert (unique IDs required)
points = []

# Add text points
for idx, (doc, vector) in enumerate(zip(docs, text_embeddings)):
    payload = {"text": doc.page_content, "media_type": "text"}
    payload.update(doc.metadata)
    # Ensure unique IDs across text and image points
    points.append(PointStruct(id=idx, vector=vector, payload=payload))

# Add image points (offset IDs by len(text_embeddings))
offset = len(text_embeddings)
for idx, (path, vector) in enumerate(zip(image_paths, image_embeddings)):
    payload = {"path": path, "media_type": "image"}
    # Ensure unique IDs across text and image points
    points.append(PointStruct(id=idx + offset, vector=vector, payload=payload))

# 7. Upsert all points into Qdrant collection
# Check if there are points to upsert
if points:
    client.upsert(collection_name=collection_name, points=points)
    print(f"Stored total {len(points)} text and image embeddings in '{collection_name}' collection.")
else:
    print("No points to store embeddings for.")

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Stored total 194 text and image embeddings in 'multimodal_collection' collection.


In [ ]:
count = client.count(collection_name=collection_name, exact=True).count
print(f"🧮 Total vectors in collection '{collection_name}': {count}")


🧮 Total vectors in collection 'multimodal_collection': 194


In [ ]:
results = client.scroll(
    collection_name=collection_name,
    limit=5  # koi bhi 5 vector entry laane ke liye
)

for item in results[0]:
    print(f"ID: {item.id}, Payload: {item.payload}")


ID: 0, Payload: {'text': "{'Product ID': '100', 'title': 'Samsung Galaxy F06 5G (Lit Violet, 128 GB) (4 GB RAM)', 'price': '₹8,499', 'category': 'Electronics > Smartphones', 'specifications': {'display': '6.7 inch HD+', 'battery': '5000 mAh', 'camera': '50MP + 2MP Rear, 8MP Front', 'processor': 'Mediatek Dimensity 6300', 'os': 'Android 15', 'ram': '4 GB', 'storage': '128 GB', 'expandable': 'Up to 1.5TB'}, 'description_path': 'Highest 5G BandsGet superb 5G connectivity with 12 5G bands and all network support. Stay", 'media_type': 'text', 'Product_ID_x': '100.json', 'Product_ID_y': '', 'Product ID': '100', 'Detail': '', 'Link': 'https://www.flipkart.com/samsung-galaxy-f06-5g-lit-violet-128-gb/p/itm140da3412b73b?pid=MOBH9AS4FA5RUHSY&lid=LSTMOBH9AS4FA5RUHSYIYVIFO&marketplace=FLIPKART&fm=neo%2Fmerchandising&iid=M_32f39bc2-1a3a-434c-a50d-8ea0817d6921_1_1BUWY8OBA8L9_MC.MOBH9AS4FA5RUHSY&ppt=browse&ppn=browse&ssid=o9hsmo7als0000001747954534726&otracker=clp_pmu_v2_Latest%2BSamsung%2Bmobiles%2B_

In [ ]:
if client.collection_exists(collection_name):
    print(f"Collection '{collection_name}' exists.")
else:
    print(f"Collection '{collection_name}' does NOT exist.")


Collection 'multimodal_collection' exists.


In [ ]:
collection_info = client.get_collection(collection_name=collection_name)
print(f"Collection '{collection_name}' info:", collection_info)


Collection 'multimodal_collection' info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=194 segments_count=1 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=512, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, w

In [ ]:
# Scroll through collection points (first 10 points)
scroll_result = client.scroll(collection_name=collection_name, limit=10)
points_list = scroll_result[0] # Get the list of points from the tuple

print(f"First 10 points in collection '{collection_name}':")
# Iterate through the list of points
for point in points_list:
    print(f"ID: {point.id}, Payload keys: {list(point.payload.keys())}")

First 10 points in collection 'multimodal_collection':
ID: 0, Payload keys: ['text', 'media_type', 'Product_ID_x', 'Product_ID_y', 'Product ID', 'Detail', 'Link', 'Gender', 'Primary Category', 'Secondary Category', 'ArticleType', 'Base Colour', 'Season', 'Year', 'Usage', 'Product Display Name', 'Product Name', 'Product Description', 'User Reviews', 'QA Threads']
ID: 1, Payload keys: ['text', 'media_type', 'Product_ID_x', 'Product_ID_y', 'Product ID', 'Detail', 'Link', 'Gender', 'Primary Category', 'Secondary Category', 'ArticleType', 'Base Colour', 'Season', 'Year', 'Usage', 'Product Display Name', 'Product Name', 'Product Description', 'User Reviews', 'QA Threads']
ID: 2, Payload keys: ['text', 'media_type', 'Product_ID_x', 'Product_ID_y', 'Product ID', 'Detail', 'Link', 'Gender', 'Primary Category', 'Secondary Category', 'ArticleType', 'Base Colour', 'Season', 'Year', 'Usage', 'Product Display Name', 'Product Name', 'Product Description', 'User Reviews', 'QA Threads']
ID: 3, Payload 

In [ ]:
info = client.get_collection(collection_name)
print(info)


status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=194 segments_count=1 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=512, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config

In [ ]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from PIL import Image

collection_name = "multimodal_collection"

embedding_model = HuggingFaceEmbeddings(model_name="clip-ViT-B-32")

# Assume client is your QdrantClient already initialized
db = Qdrant(
    client=client,
    collection_name=collection_name,
    embeddings=embedding_model,
    content_payload_key="text",
    metadata_payload_key="metadata"
)

<ipython-input-27-8c19953b0c95>:10: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  db = Qdrant(


🔹 4. QUERY FLOW: EMBEDDING → RETRIEVAL → GENERATION


In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 3})  # top 3 results
query="electronic material"
results = retriever.get_relevant_documents(query)
print(f"Retrieved {len(results)} documents")
for r in results:
    print(r.metadata, r.page_content)


<ipython-input-28-4458723b981d>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


Retrieved 3 documents
{'_id': 42, '_collection_name': 'multimodal_collection'} Cable', 'Adapter'], 'dimensions_cm': {'length': 2.3, 'width': 5.9, 'height': 6.3}, 'weight_g': 300, 'material': 'Plastic', 'battery_type': 'Super Capacitor', 'temperature_tolerance': '-20°C to 85°C', 'warranty': '1 Year', 'installation': {'available': True, 'charges': '₹1200 + GST (paid to technician)'}, 'app_compatibility': ['iOS', 'Android'], 'app_name': 'Qubo Pro App', 'live_streaming': True, 'download_videos': True, 'country_of_origin': 'India', 'manufacturer': 'Hero Electronix Pvt. Ltd.',
{'_id': 52, '_collection_name': 'multimodal_collection'} {'overall': None, 'category': '#880 in Routers & Networking', 'subcategory': '#200,126 in Computers & Accessories'} B0DJP3ZZN7
{'_id': 79, '_collection_name': 'multimodal_collection'} Width)', 'dimensions_cm': {'length': 40.6, 'width': 231, 'height': 193}, 'item_depth_inches': 16, 'weight_kg': 80, 'style': 'Modern', 'assembly_required': True, 'shape': 'Rectangula

In [ ]:
def embed_text_image(text: str, image_path: str):
    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(text=[text], images=image, return_tensors="pt", padding=True)

    # Separate inputs for text and image
    text_inputs = {key: inputs[key] for key in inputs if key in ['input_ids', 'attention_mask']}
    image_inputs = {key: inputs[key] for key in inputs if key in ['pixel_values']}

    with torch.no_grad():
        # Pass only text-specific inputs to get_text_features
        text_emb = clip_model.get_text_features(**text_inputs)
        # Pass only image-specific inputs to get_image_features
        image_emb = clip_model.get_image_features(**image_inputs)

    combined_emb = (text_emb + image_emb) / 2
    return combined_emb[0].numpy()

In [ ]:
from langchain.schema import Document

def custom_retrieve_from_vector(vector, k=3):
    hits = client.search(
        collection_name="multimodal_collection",
        query_vector=vector.tolist(),
        limit=k,
        with_payload=True
    )
    docs = []
    for hit in hits:
        text = hit.payload.get("text", "")
        metadata = hit.payload.get("metadata", {})
        docs.append(Document(page_content=text, metadata=metadata))
    return docs


In [ ]:
def format_docs_for_prompt(docs):
    return "\n\n".join([doc.page_content for doc in docs])


Setup of a model

In [ ]:
from langchain.chat_models import ChatOpenAI
from google.colab import userdata # Import userdata to access secrets

# Access the stored OpenAI API key from Colab secrets
openai_api_key = userdata.get('OPEN_API_KEY')


In [ ]:
!pip install transformers


 Step 4: Pipeline +Format as context and send to LLM (GPT2 / GPT-3.5)



In [ ]:
from transformers import pipeline
from google.colab import userdata  # for accessing stored secrets

# Load the Hugging Face model pipeline (e.g., "gpt2" or hosted on HF)
api_token = userdata.get('HUGGINGFACEHUB_API_TOKEN')

# You can use a hosted model (e.g., OpenAssistant, Mistral, LLaMA) or GPT2 (local)
generator = pipeline("text-generation", model="gpt2", tokenizer="gpt2")


def format_docs_for_prompt(context_docs):
    return "\n\n".join(
        f"Title: {doc.metadata.get('title', '')}\n"
        f"Price: {doc.metadata.get('price', '')}\n"
        f"Brand: {doc.metadata.get('brand', '')}\n"
        f"Details: {doc.page_content}"
        for doc in context_docs
    )


def run_llm(query_text, context_docs):
    context = format_docs_for_prompt(context_docs)
    prompt = f"""[INST] You are a helpful shopping assistant AI. Based on the context, answer the query by extracting:
- Product Title
- Price
- Key Specifications
- Brand
- Unique Features
- Recommendation summary

Context:
{context}

Context:
{context}

Question:
{query_text}

Answer:"""
    response = generator(prompt, max_new_tokens=400, do_sample=True, temperature=0.7)
    return response[0]["generated_text"].replace(prompt, "").strip()


Device set to use cuda:0


In [ ]:
'''from langchain.chat_models import ChatOpenAI
from google.colab import userdata # Import userdata to access secrets

# Access the stored OpenAI API key from Colab secrets
openai_api_key = userdata.get('OPEN_API_KEY')

# Initialize ChatOpenAI by passing the API key directly
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

def run_llm(query_text, context_docs):
    context = format_docs_for_prompt(context_docs)
    prompt = f"""You are an assistant for answering queries based on the given context.

Context:
{context}

Question:
{query_text}

Answer:"""
    return llm.predict(prompt)'''

<ipython-input-33-5ec9d86fc792>:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)


Build a Multimodal_rag system

In [ ]:
def multimodal_rag(query_text, image_path):
    embedded_vector = embed_text_image(query_text, image_path)
    retrieved_docs = custom_retrieve_from_vector(embedded_vector, k=3)
    response = run_llm(query_text, retrieved_docs)
    return response


Output 

In [ ]:
import os

query_text = "Are these shoes waterproof and good for running?"
image_path = "/content/Screenshot 2025-05-27 024222.png"

# Check if the file exists at the specified path
if not os.path.exists(image_path):
    print(f"Error: The image file was not found at {image_path}")
    # You can add logic here to:
    # - Print a list of files in the directory to help debug
    # - Ask the user for the correct path
    # - Skip this particular query if the image is essential
    # For now, we will just print the error and the code will likely fail again
    # if the path is still incorrect.
else:
    answer = multimodal_rag(query_text, image_path)
    print("Answer:", answer)

<ipython-input-30-02e4a0060559>:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: Yes, they are.

Question:

Does this really mean there is a waterproof version?

Answer:

Yes, very well.

Question:

What is the difference between a waterproof and a waterproof version?

Answer:

The difference is in the way the shoe is made. The waterproof is made of a non-toxic material. The waterproof version is made of a non-toxic material and the waterproof version is made of an odorless material.

Question:

Is this the only waterproof shoe in the UK?

Answer:

Yes, because it is an 'on-brand' brand.

Question:

What is the difference between a waterproof and a waterproof version?

Answer:

The difference is in the way the shoe is made. The waterproof is made of a non-toxic material. The waterproof version is made of a non-toxic material and the waterproof version is made of an odorless material.

Question:

What is the difference between a waterproof and a waterproof version?

Answer:

The difference is in the way the shoe is made. The waterproof is made of a non-toxic